In [103]:
# 혼자하는 머신러닝 학습 2020.08.15
# 단톡방에도 공유를 했는데, '문자를 인코딩하고 딕셔너리의 0과 1로 매핑하기'에 대한 추가학습이 필요!

In [139]:
import pandas as pd
import numpy as np 
# 신용카드 사기 검출
card_df = pd.read_csv('./creditcard.csv')
card_df.head()
# Time: Elapsed between transactions, drop-out
# Amount: Transaction
# Class: 0-normal, 1-abnormal

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [140]:
from sklearn.model_selection import train_test_split
# Time: drop-out
def preprocessed_df_origin(df=None):
    df_copy = df.copy()
    df_copy.drop('Time', axis=1, inplace=True)
    return df_copy

def train_test_df_origin(df=None):
    df_copy = preprocessed_df_origin(df)
    # target == Class
    X_features = df_copy.iloc[:, :-1]
    Y_target = df_copy.iloc[:, -1]
    X_train, X_test, Y_train, Y_test = train_test_split(X_features, Y_target, test_size=0.3, random_state=1, stratify=Y_target)
    return X_train, X_test, Y_train, Y_test

In [141]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
# Evaluation Function
def ML_score(Y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(Y_test, pred)
    accuracy = accuracy_score(Y_test, pred)
    precision = precision_score(Y_test, pred)
    recall = recall_score(Y_test, pred)
    f1 = f1_score(Y_test, pred)
    roc_auc = roc_auc_score(Y_test, pred_proba)
    print('Confusion Matrix\n', confusion)
    print('Acc = {0:.4f}, Prec = {1:.4f}, Recall = {2:.4f}, F1 = {3:.4f}, AUC = {4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [142]:
from sklearn.linear_model import LogisticRegression
def ML_tester(model, in_train=None, in_test=None, out_train=None, out_test=None):
    model.fit(in_train, out_train)
    pred = model.predict(in_test)
    pred_proba = model.predict_proba(in_test)[:, 1]
    # 모델평가
    ML_score(out_test, pred, pred_proba)

In [143]:
card_df['Amount'].value_counts()
# 이상치 확인

1.00       13688
1.98        6044
0.89        4872
9.99        4747
15.00       3280
           ...  
192.63         1
218.84         1
195.52         1
793.50         1
1080.06        1
Name: Amount, Length: 32767, dtype: int64

In [144]:
corr = card_df.corr(method = 'pearson').iloc[:,-1]
# 피어슨 상관관계 분석, 1에 가까울수록 강한 상관관계
corr

Time     -0.012323
V1       -0.101347
V2        0.091289
V3       -0.192961
V4        0.133447
V5       -0.094974
V6       -0.043643
V7       -0.187257
V8        0.019875
V9       -0.097733
V10      -0.216883
V11       0.154876
V12      -0.260593
V13      -0.004570
V14      -0.302544
V15      -0.004223
V16      -0.196539
V17      -0.326481
V18      -0.111485
V19       0.034783
V20       0.020090
V21       0.040413
V22       0.000805
V23      -0.002685
V24      -0.007221
V25       0.003308
V26       0.004455
V27       0.017580
V28       0.009536
Amount    0.005632
Class     1.000000
Name: Class, dtype: float64

In [145]:
def dropout_df(df=None, column=None, weight=1.5):
    # column == Amount2
    fraud = df[df['Class']==1][column]
    quarter_25 = np.percentile(fraud.values, 25)
    quarter_75 = np.percentile(fraud.values, 75) 
    IQR_BOX = (quarter_75 - quarter_25) * weight
    min = quarter_25 - IQR_BOX
    max = quarter_75 + IQR_BOX
    dropout = fraud[(fraud < min) | (fraud > max)].index
    # return DataFrame Index
    return dropout

In [146]:
# 이상치 제거
def preprocessed_df(df=None):
    df_copy = df.copy()
    amount_scaler = np.log1p(df_copy['Amount'])
    df_copy.insert(0, 'newAmount', amount_scaler)
    df_copy.drop(['Time','Amount'], axis=1, inplace=True)
    index = dropout_df(df_copy, 'V17', 1.5)
    df_copy.drop(index, axis=0, inplace=True)
    return df_copy

def train_test_df(df=None):
    df_copy = preprocessed_df(df)
    # target == Class
    X_features = df_copy.iloc[:, :-1]
    Y_target = df_copy.iloc[:, -1]
    X_train, X_test, Y_train, Y_test = train_test_split(X_features, Y_target, test_size=0.3, random_state=1, stratify=Y_target)
    return X_train, X_test, Y_train, Y_test

In [147]:
# 분류 모델인 로지스틱 회귀 로드!
# 로지스틱 회귀는 텍스트 분류에서 뛰어난 성능!
classifier = LogisticRegression()
print("초기 모델 평가")
X_train, X_test, Y_train, Y_test = train_test_df_origin(card_df)
ML_tester(classifier, X_train, X_test, Y_train, Y_test)
print("\n이상치 제거 후 모델 평가")
X_train, X_test, Y_train, Y_test = train_test_df(card_df)
ML_tester(classifier, X_train, X_test, Y_train, Y_test)

초기 모델 평가
Confusion Matrix
 [[85280    15]
 [   48   100]]
Acc = 0.9993, Prec = 0.8696, Recall = 0.6757, F1 = 0.7605, AUC = 0.9634

이상치 제거 후 모델 평가
Confusion Matrix
 [[85284    11]
 [   45   103]]
Acc = 0.9993, Prec = 0.9035, Recall = 0.6959, F1 = 0.7863, AUC = 0.9790
